In [5]:
!pip install pandas numpy

  Using cached pandas-2.2.3-cp39-cp39-macosx_11_0_arm64.whl (11.3 MB)
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl (5.3 MB)
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
You should consider upgrading via the '/Users/hyejiyu/.pyenv/versions/3.9.6/envs/data-capstone-model/bin/python -m pip install --upgrade pip' command.


In [12]:
import pandas as pd
import numpy as np
import ast
import re

In [55]:
# df = pd.read_csv("hatespeech_unsmile_hatescore.csv")
df = pd.read_csv("processed_data.csv")
df.head()

,sentence,label,one_hot_label
0,자한당틀딱들 악플질 고만해라.,"['정치성향차별', '연령차별']","[0, 1, 0, 1, 0, 0, 0, 0, 0]"
1,정치적으로 편향된 평론한은 분은 별로,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
2,적당히좀 쳐먹지.그랬냐 안그래도 문재인 때문에 나라 엉망진창인데,['정치성향차별'],"[0, 0, 0, 1, 0, 0, 0, 0, 0]"
3,안서는 아재들 풀발기,['연령차별'],"[0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,우와 능력자,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [56]:
# 문자열을 리스트로 변환
df["label"] = df["label"].apply(ast.literal_eval)

In [57]:
# '혐오욕설' 제거 함수 정의
def remove_hate_speech(labels):
    # '혐오욕설'이 포함되어 있고 다른 항목도 있을 때 '혐오욕설' 제거
    if "혐오욕설" in labels and len(labels) > 1:
        labels.remove("혐오욕설")
    return labels


# '혐오욕설' 제거 적용
df["label"] = df["label"].apply(remove_hate_speech)

df.head()

,sentence,label,one_hot_label
0,자한당틀딱들 악플질 고만해라.,"[정치성향차별, 연령차별]","[0, 1, 0, 1, 0, 0, 0, 0, 0]"
1,정치적으로 편향된 평론한은 분은 별로,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
2,적당히좀 쳐먹지.그랬냐 안그래도 문재인 때문에 나라 엉망진창인데,[정치성향차별],"[0, 0, 0, 1, 0, 0, 0, 0, 0]"
3,안서는 아재들 풀발기,[연령차별],"[0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,우와 능력자,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [58]:
# 라벨 목록 정의
label_list = [
    "성차별",
    "연령차별",
    "외모차별",
    "정치성향차별",
    "출신차별",
    "인종차별",
    "종교차별",
    "혐오욕설",
    "clean",
]

In [59]:
# 원핫인코딩 함수 정의
def one_hot_encode(labels):
    # 모든 라벨에 대해 0으로 초기화된 리스트 생성
    one_hot = [0] * len(label_list)

    # 빈 배열인 경우 clean 라벨 활성화
    if len(labels) == 0:
        clean_index = label_list.index("clean")
        one_hot[clean_index] = 1
        return one_hot

    # 해당하는 라벨들을 1로 설정
    for label in labels:
        if label in label_list:
            index = label_list.index(label)
            one_hot[index] = 1

    return one_hot


# 원핫인코딩 적용
df["one_hot_label"] = df["label"].apply(one_hot_encode)

# 결과 확인
df.head()

,sentence,label,one_hot_label
0,자한당틀딱들 악플질 고만해라.,"[정치성향차별, 연령차별]","[0, 1, 0, 1, 0, 0, 0, 0, 0]"
1,정치적으로 편향된 평론한은 분은 별로,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
2,적당히좀 쳐먹지.그랬냐 안그래도 문재인 때문에 나라 엉망진창인데,[정치성향차별],"[0, 0, 0, 1, 0, 0, 0, 0, 0]"
3,안서는 아재들 풀발기,[연령차별],"[0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,우와 능력자,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [60]:
# 전처리할 대상 컬럼 이름
target_column = "sentence"


# 전처리 함수 정의
def clean_text(text):
    if pd.isnull(text):
        return text
    # 1. ", ~, ; 제거
    text = re.sub(r"[\"'~;:(),!^/♡*=+&%<>♥]", "", text)
    # 2. 연속된 마침표 제거
    text = re.sub(r"\.{2,}", "", text)
    # 3. 연속된 물음표 제거
    text = re.sub(r"\?{2,}", "", text)
    # 5. 한글 자모(초성, 중성, 종성 포함) 제거
    text = re.sub(r"[ㄱ-ㅎㅏ-ㅣ]+", "", text)
    text = re.sub(r"\s{2,}", " ", text)  # 중복 공백 제거

    return text.strip()

# 특정 컬럼에 전처리 적용
df[target_column] = df[target_column].apply(clean_text)

In [61]:
df.head(10)

,sentence,label,one_hot_label
0,자한당틀딱들 악플질 고만해라.,"[정치성향차별, 연령차별]","[0, 1, 0, 1, 0, 0, 0, 0, 0]"
1,정치적으로 편향된 평론한은 분은 별로,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
2,적당히좀 쳐먹지.그랬냐 안그래도 문재인 때문에 나라 엉망진창인데,[정치성향차별],"[0, 0, 0, 1, 0, 0, 0, 0, 0]"
3,안서는 아재들 풀발기,[연령차별],"[0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,우와 능력자,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
5,맛녀석 콩트보다 약했음맛녀석 애청자로써 70실력발휘,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
6,주영훈 솔직히 호감임 잉꼬부부로 소문났잖아,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
7,이게주간아이돌이랑머가달라,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
8,아오 슈박 회사생활도 졑깥고 돈벌기 힘들어 죽겠구만 뭔 저딴것들 자꾸 tv나와서 사...,[혐오욕설],"[0, 0, 0, 0, 0, 0, 0, 1, 0]"
9,문재인 하는게 뭐 별거있냐?가 하는짓인데 어련하겠어.,[정치성향차별],"[0, 0, 0, 1, 0, 0, 0, 0, 0]"


In [62]:
df.to_csv("processed_data.csv", index=False)

In [63]:
df['sentence'].apply(type).value_counts()

sentence
<class 'str'>      139191
<class 'float'>        67
Name: count, dtype: int64

In [68]:
df2 = pd.read_csv("./processed_data.csv")
df2.head()

,sentence,label,one_hot_label
0,자한당틀딱들 악플질 고만해라.,"['정치성향차별', '연령차별']","[0, 1, 0, 1, 0, 0, 0, 0, 0]"
1,정치적으로 편향된 평론한은 분은 별로,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
2,적당히좀 쳐먹지.그랬냐 안그래도 문재인 때문에 나라 엉망진창인데,['정치성향차별'],"[0, 0, 0, 1, 0, 0, 0, 0, 0]"
3,안서는 아재들 풀발기,['연령차별'],"[0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,우와 능력자,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [69]:
# 원래 라벨 리스트
label_list = [
    "성차별",
    "연령차별",
    "외모차별",
    "정치성향차별",
    "출신차별",
    "인종차별",
    "종교차별",
    "혐오욕설",
    "clean",
]

# 재분류 기준
reclassified_labels = {
    "성차별": "성차별",
    "연령차별": "연령차별",
    "혐오욕설": "혐오욕설",
    "clean": "clean",
    "default": "기타혐오",  # 나머지 라벨은 "기타혐오"로 분류
}

# 재분류 함수
def reclassify_labels(labels):
    label_list = eval(labels) if labels.strip() else []
    if not label_list:  # 라벨이 비어 있으면 "clean"으로 처리
        return ["clean"]
    reclassified = set()
    for label in label_list:  # 문자열로 저장된 리스트를 실제 리스트로 변환
        reclassified.add(reclassified_labels.get(label, "기타혐오"))
    return list(reclassified)

# 재분류된 라벨 컬럼 추가
df2["reclassified_label"] = df2["label"].apply(reclassify_labels)

# 재분류된 원핫 인코딩 생성
new_label_list = ["성차별", "연령차별", "혐오욕설", "clean", "기타혐오"]

def create_one_hot(labels):
    one_hot = [1 if label in labels else 0 for label in new_label_list]
    return one_hot

df2["one_hot_label_reclassified"] = df2["reclassified_label"].apply(create_one_hot)

# 결과 저장
df2.to_csv("reclassified_test.csv", index=False)

In [70]:
df2.head(10)

,sentence,label,one_hot_label,reclassified_label,one_hot_label_reclassified
0,자한당틀딱들 악플질 고만해라.,"['정치성향차별', '연령차별']","[0, 1, 0, 1, 0, 0, 0, 0, 0]","[연령차별, 기타혐오]","[0, 1, 0, 0, 1]"
1,정치적으로 편향된 평론한은 분은 별로,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]",[clean],"[0, 0, 0, 1, 0]"
2,적당히좀 쳐먹지.그랬냐 안그래도 문재인 때문에 나라 엉망진창인데,['정치성향차별'],"[0, 0, 0, 1, 0, 0, 0, 0, 0]",[기타혐오],"[0, 0, 0, 0, 1]"
3,안서는 아재들 풀발기,['연령차별'],"[0, 1, 0, 0, 0, 0, 0, 0, 0]",[연령차별],"[0, 1, 0, 0, 0]"
4,우와 능력자,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]",[clean],"[0, 0, 0, 1, 0]"
5,맛녀석 콩트보다 약했음맛녀석 애청자로써 70실력발휘,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]",[clean],"[0, 0, 0, 1, 0]"
6,주영훈 솔직히 호감임 잉꼬부부로 소문났잖아,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]",[clean],"[0, 0, 0, 1, 0]"
7,이게주간아이돌이랑머가달라,[],"[0, 0, 0, 0, 0, 0, 0, 0, 1]",[clean],"[0, 0, 0, 1, 0]"
8,아오 슈박 회사생활도 졑깥고 돈벌기 힘들어 죽겠구만 뭔 저딴것들 자꾸 tv나와서 사...,['혐오욕설'],"[0, 0, 0, 0, 0, 0, 0, 1, 0]",[혐오욕설],"[0, 0, 1, 0, 0]"
9,문재인 하는게 뭐 별거있냐?가 하는짓인데 어련하겠어.,['정치성향차별'],"[0, 0, 0, 1, 0, 0, 0, 0, 0]",[기타혐오],"[0, 0, 0, 0, 1]"


In [76]:
df2.shape

(139256, 5)

In [77]:
df2 = df2[df2["sentence"].notnull()]
df2.shape

(139190, 5)

In [78]:
df2.to_csv("reclassified_test.csv", index=False)